In [14]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support, confusion_matrix,
    precision_recall_curve, roc_curve, auc, classification_report
)
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
import re
import emoji
from transformers import (
    DistilBertForSequenceClassification, 
    AutoTokenizer,
    Trainer, 
    TrainingArguments,
    EarlyStoppingCallback
)
from transformers.integrations import WandbCallback
import optuna
from lime.lime_text import LimeTextExplainer
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

# Check for MPS (Apple Silicon) device
use_mps = torch.backends.mps.is_available()
device = torch.device('mps' if use_mps else 'cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Model configuration
MODEL_NAME = "distilbert-base-uncased"
NUM_LABELS = 2  # Binary classification
OUTPUT_DIR = "./results"
MODEL_DIR = "./saved_model"
MAX_LENGTH = 128

# Text preprocessing
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Replace URLs with token
    text = re.sub(r'https?://\S+|www\.\S+', '[URL]', text)
    
    # Replace user mentions with token
    text = re.sub(r'@\w+', '[USER]', text)
    
    # Replace hashtags with token but keep the text
    text = re.sub(r'#(\w+)', r'\1', text)
    
    # Convert emojis to text
    text = emoji.demojize(text)
    
    # Replace repeated characters (e.g., "coooool" -> "cool")
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Data preparation
def prepare_data(df):
    # Apply preprocessing to tweets
    df['tweet'] = df['tweet'].astype(str).apply(preprocess_text)
    df['label'] = df['class'].apply(lambda x: 0 if x == 2 else 1)
    print(df['label'].value_counts())
    df.drop(columns=['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither', 'class'], inplace=True)

    # Calculate class weights for imbalanced dataset
    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(df["label"]),
        y=df["label"]
    )
    
    class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}
    print(f"Class weights: {class_weight_dict}")
    
    return df, class_weight_dict

# Memory-efficient dataset implementation
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __getitem__(self, idx):
        # Tokenize on-the-fly instead of storing all tokenized data in memory
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt"
        )
        
        # Remove batch dimension added by tokenizer when return_tensors="pt"
        encoding = {k: v.squeeze(0) for k, v in encoding.items()}
        
        encoding["labels"] = torch.tensor(self.labels[idx])
        return encoding
    
    def __len__(self):
        return len(self.labels)

# Define evaluation metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = torch.nn.functional.softmax(torch.tensor(logits), dim=-1).numpy()
    predictions = np.argmax(logits, axis=-1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, 
        predictions, 
        average="binary", 
        zero_division=0
    )
    
    accuracy = accuracy_score(labels, predictions)
    
    tn, fp, fn, tp = confusion_matrix(labels, predictions).ravel()
    
    # Calculate ROC AUC
    fpr, tpr, _ = roc_curve(labels, probs[:, 1])
    roc_auc = auc(fpr, tpr)
    
    return {
        "accuracy": accuracy,
        "f1": f1,
        "precision": precision,
        "recall": recall,
        "true_positives": tp,
        "false_negatives": fn,
        "false_positives": fp,
        "true_negatives": tn,
        "roc_auc": roc_auc
    }

# Hyperparameter optimization using Optuna
def objective(trial):
    # Define hyperparameters to search
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    weight_decay = trial.suggest_float("weight_decay", 0.001, 0.1, log=True)
    warmup_ratio = trial.suggest_float("warmup_ratio", 0.05, 0.2)
    batch_size = trial.suggest_categorical("batch_size", [8, 16])
    
    # Define training arguments for this trial
    training_args = TrainingArguments(
        output_dir=f"{OUTPUT_DIR}/trial_{trial.number}",
        num_train_epochs=3,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size * 2,
        gradient_accumulation_steps=4,
        warmup_ratio=warmup_ratio,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        logging_dir=f"./logs/trial_{trial.number}",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,
        report_to="none",
        dataloader_num_workers=0,
        seed=RANDOM_SEED,
        optim="adamw_torch"
    )
    
    # Initialize model for this trial
    model = DistilBertForSequenceClassification.from_pretrained(
        MODEL_NAME, 
        num_labels=NUM_LABELS,
        id2label={0: "not offensive", 1: "offensive"}
    )
    
    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
    )
    
    # Train model
    trainer.train()
    
    # Evaluate model
    metrics = trainer.evaluate()
    
    return metrics["eval_f1"]

# Custom threshold prediction function
def predict_with_threshold(model, tokenizer, text, threshold=0.5):
    # Preprocess text
    processed_text = preprocess_text(text)
    
    # Tokenize
    inputs = tokenizer(
        processed_text, 
        return_tensors="pt", 
        truncation=True, 
        max_length=MAX_LENGTH,
        padding="max_length"
    )
    
    # Move inputs to the right device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Get prediction
    with torch.inference_mode():
        outputs = model(**inputs)
    
    logits = outputs.logits.cpu()
    probabilities = torch.softmax(logits, dim=-1)
    
    # Apply custom threshold
    prediction = 1 if probabilities[0][1].item() > threshold else 0
    
    return {
        "text": text,
        "processed_text": processed_text,
        "class_name": model.config.id2label[prediction],
        "confidence": probabilities[0][prediction].item(),
        "offensive_probability": probabilities[0][1].item()
    }

# Error analysis function
def analyze_errors(model, tokenizer, texts, true_labels, threshold=0.5):
    results = []
    
    for text, true_label in zip(texts, true_labels):
        prediction = predict_with_threshold(model, tokenizer, text, threshold)
        pred_label = 1 if prediction["offensive_probability"] > threshold else 0
        
        if pred_label != true_label:
            results.append({
                "text": text,
                "processed_text": prediction["processed_text"],
                "true_label": true_label,
                "predicted_label": pred_label,
                "offensive_probability": prediction["offensive_probability"]
            })
    
    return pd.DataFrame(results)

# Model explainability function
def explain_prediction(model, tokenizer, text, explainer):
    processed_text = preprocess_text(text)
    
    def predict_fn(texts):
        results = []
        for t in texts:
            inputs = tokenizer(
                t,
                return_tensors="pt",
                truncation=True,
                max_length=MAX_LENGTH,
                padding="max_length"
            )
            inputs = {k: v.to(device) for k, v in inputs.items()}
            
            with torch.inference_mode():
                outputs = model(**inputs)
            
            logits = outputs.logits.cpu().numpy()
            probs = torch.nn.functional.softmax(torch.tensor(logits), dim=-1).numpy()
            results.append(probs)
        
        return np.vstack(results)
    
    # Generate explanation
    exp = explainer.explain_instance(
        processed_text, 
        predict_fn, 
        num_features=10, 
        num_samples=1000,
        labels=(1,)  # Explain offensive class
    )
    
    return exp

# Find optimal classification threshold
def find_optimal_threshold(model, tokenizer, texts, true_labels):
    probabilities = []
    
    for text in texts:
        prediction = predict_with_threshold(model, tokenizer, text, threshold=0.5)
        probabilities.append(prediction["offensive_probability"])
    
    # Calculate precision-recall curve
    precision, recall, thresholds = precision_recall_curve(true_labels, probabilities)
    
    # Calculate F1 score for each threshold
    f1_scores = 2 * precision * recall / (precision + recall + 1e-10)
    
    # Find threshold with highest F1 score
    best_threshold_idx = np.argmax(f1_scores)
    best_threshold = thresholds[best_threshold_idx]
    
    # Plot precision-recall curve
    plt.figure(figsize=(10, 7))
    plt.plot(recall, precision, marker='.')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve (Best threshold: {best_threshold:.3f})')
    plt.grid(True)
    plt.savefig('precision_recall_curve.png')
    
    return best_threshold

# Main training pipeline
def main(df):
    # Prepare data
    processed_df, class_weights = prepare_data(df)
    
    # Create stratified k-fold cross-validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
    fold_results = []
    
    # Initialize tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(processed_df["tweet"], processed_df["label"])):
        print(f"Training fold {fold+1}/5...")
        
        # Split data
        train_texts = processed_df["tweet"].iloc[train_idx].reset_index(drop=True)
        train_labels = processed_df["label"].iloc[train_idx].reset_index(drop=True).to_numpy()
        val_texts = processed_df["tweet"].iloc[val_idx].reset_index(drop=True)
        val_labels = processed_df["label"].iloc[val_idx].reset_index(drop=True).to_numpy()
        
        # Create datasets
        global train_dataset, val_dataset  # Make them accessible to the objective function
        train_dataset = TweetDataset(train_texts, train_labels, tokenizer, MAX_LENGTH)
        val_dataset = TweetDataset(val_texts, val_labels, tokenizer, MAX_LENGTH)
        
        # Hyperparameter optimization if this is first fold
        if fold == 0:
            print("Optimizing hyperparameters...")
            study = optuna.create_study(direction="maximize")
            study.optimize(objective, n_trials=5)  # Reduced trials for limited compute
            
            best_params = study.best_params
            print(f"Best hyperparameters: {best_params}")
        
        # Train with best hyperparameters
        training_args = TrainingArguments(
            output_dir=f"{OUTPUT_DIR}/fold_{fold}",
            num_train_epochs=5,
            per_device_train_batch_size=best_params["batch_size"],
            per_device_eval_batch_size=best_params["batch_size"] * 2,
            gradient_accumulation_steps=4,
            warmup_ratio=best_params["warmup_ratio"],
            learning_rate=best_params["learning_rate"],
            weight_decay=best_params["weight_decay"],
            logging_dir=f"./logs/fold_{fold}",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            save_total_limit=1,
            load_best_model_at_end=True,
            metric_for_best_model="f1",
            greater_is_better=True,
            report_to="none",
            dataloader_num_workers=0,
            seed=RANDOM_SEED,
            optim="adamw_torch"
        )
        
        # Initialize model with class weights
        model = DistilBertForSequenceClassification.from_pretrained(
            MODEL_NAME, 
            num_labels=NUM_LABELS,
            id2label={0: "not offensive", 1: "offensive"}
        )
        
        # Set class weights in model config
        model.config.class_weights = class_weights
        
        # Initialize trainer with early stopping
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=compute_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
        )
        
        # Train model
        trainer.train()
        
        # Evaluate model
        metrics = trainer.evaluate()
        fold_results.append(metrics)
        
        # Save model for this fold
        trainer.save_model(f"{MODEL_DIR}/fold_{fold}")
        
        # Find optimal threshold
        optimal_threshold = find_optimal_threshold(
            model, tokenizer, val_texts, val_labels
        )
        print(f"Optimal threshold for fold {fold+1}: {optimal_threshold:.3f}")
        
        # Error analysis on validation set
        error_df = analyze_errors(
            model, tokenizer, val_texts, val_labels, threshold=optimal_threshold
        )
        
        # Save error analysis
        if not error_df.empty:
            error_df.to_csv(f"error_analysis_fold_{fold}.csv", index=False)
            print(f"Saved error analysis for fold {fold+1} with {len(error_df)} misclassified examples")
        
        # Create LIME explainer
        explainer = LimeTextExplainer(class_names=["not offensive", "offensive"])
        
        # Explain some misclassifications
        if not error_df.empty:
            # Get up to 5 examples of each error type (FP and FN)
            false_positives = error_df[error_df["true_label"] == 0].head(5)
            false_negatives = error_df[error_df["true_label"] == 1].head(5)
            
            # Explain false positives
            print("\nExplaining false positives:")
            for idx, row in false_positives.iterrows():
                exp = explain_prediction(model, tokenizer, row["text"], explainer)
                print(f"\nText: {row['text']}")
                print("Features contributing to 'offensive' classification:")
                exp.as_list(label=1)
                
            # Explain false negatives
            print("\nExplaining false negatives:")
            for idx, row in false_negatives.iterrows():
                exp = explain_prediction(model, tokenizer, row["text"], explainer)
                print(f"\nText: {row['text']}")
                print("Features contributing to 'offensive' classification:")
                exp.as_list(label=1)
        
    # Average results across folds
    avg_results = {
        metric: np.mean([fold[f"eval_{metric}"] for fold in fold_results])
        for metric in fold_results[0].keys() if metric.startswith("eval_")
    }
    
    print("\nAverage results across folds:")
    for metric, value in avg_results.items():
        print(f"{metric}: {value:.4f}")
    
    # Train final model on all data
    print("\nTraining final model on all data...")
    
    # Create datasets
    all_texts = processed_df["tweet"].reset_index(drop=True)
    all_labels = processed_df["label"].reset_index(drop=True).to_numpy()
    full_dataset = TweetDataset(all_texts, all_labels, tokenizer, MAX_LENGTH)
    
    # Training arguments for final model
    final_training_args = TrainingArguments(
        output_dir=f"{OUTPUT_DIR}/final",
        num_train_epochs=5,
        per_device_train_batch_size=best_params["batch_size"],
        per_device_eval_batch_size=best_params["batch_size"] * 2,
        gradient_accumulation_steps=4,
        warmup_ratio=best_params["warmup_ratio"],
        learning_rate=best_params["learning_rate"],
        weight_decay=best_params["weight_decay"],
        logging_dir="./logs/final",
        save_strategy="epoch",
        save_total_limit=1,
        report_to="none",
        dataloader_num_workers=0,
        seed=RANDOM_SEED,
        optim="adamw_torch"
    )
    
    # Initialize final model
    final_model = DistilBertForSequenceClassification.from_pretrained(
        MODEL_NAME, 
        num_labels=NUM_LABELS,
        id2label={0: "not offensive", 1: "offensive"}
    )
    
    # Set class weights in model config
    final_model.config.class_weights = class_weights
    
    # Initialize trainer
    final_trainer = Trainer(
        model=final_model,
        args=final_training_args,
        train_dataset=full_dataset
    )
    
    # Train final model
    final_trainer.train()
    
    # Save final model and tokenizer
    final_trainer.save_model(MODEL_DIR)
    tokenizer.save_pretrained(MODEL_DIR)
    
    # Return final model and tokenizer
    return final_model, tokenizer, optimal_threshold

# Function to load saved model and make predictions
def load_model_and_predict(text, threshold=None):
    # Load model and tokenizer
    model = DistilBertForSequenceClassification.from_pretrained(MODEL_DIR)
    tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
    
    # Use default threshold if none provided
    if threshold is None:
        # Try to load threshold from config
        if hasattr(model.config, "threshold"):
            threshold = model.config.threshold
        else:
            threshold = 0.5
    
    # Make prediction
    return predict_with_threshold(model, tokenizer, text, threshold)

# Usage example:


Using device: mps


In [15]:
df = pd.read_csv("../content-moderation/data/labeled_data.csv")

# Train model
model, tokenizer, optimal_threshold = main(df)

# Save optimal threshold in model config
model.config.threshold = optimal_threshold
model.save_pretrained(MODEL_DIR)

# Example usage
sample_tweets = [
    "I love this new app! It's amazing!",
    "You are so stupid, I hate you",
    "The weather is nice today :)",
    "@user This is completely unacceptable behavior #angry"
]

print("\nTesting final model on sample tweets:")
for tweet in sample_tweets:
    result = predict_with_threshold(model, tokenizer, tweet, threshold=optimal_threshold)
    print(f"Tweet: {tweet}")
    print(f"Prediction: {result['class_name']} (confidence: {result['confidence']:.4f})")
    print(f"Offensive probability: {result['offensive_probability']:.4f}")
    print()

[I 2025-05-05 19:48:03,092] A new study created in memory with name: no-name-13a707c9-a70b-4182-abb3-b0dc589562e6


label
1    20620
0     4163
Name: count, dtype: int64
Class weights: {0: np.float64(2.9765793898630797), 1: np.float64(0.6009456838021339)}
Training fold 1/5...
Optimizing hyperparameters...


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,True Positives,False Negatives,False Positives,True Negatives,Roc Auc
0,0.187300,0.118990,0.962074,0.977112,0.981174,0.973084,4013,111,77,756,0.986760
1,0.079800,0.127992,0.962074,0.977328,0.972169,0.982541,4052,72,116,717,0.988772
2,0.066400,0.135917,0.963889,0.978358,0.975645,0.981086,4046,78,101,732,0.987982


[I 2025-05-05 20:26:40,752] Trial 0 finished with value: 0.9783581187280861 and parameters: {'learning_rate': 2.512256003028559e-05, 'weight_decay': 0.006625771682625362, 'warmup_ratio': 0.06075416218145075, 'batch_size': 8}. Best is trial 0 with value: 0.9783581187280861.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,True Positives,False Negatives,False Positives,True Negatives,Roc Auc
1,No log,0.109293,0.960460,0.976179,0.978558,0.973812,4016,108,88,745,0.986124
2,0.166900,0.124124,0.957232,0.974556,0.964829,0.984481,4060,64,148,685,0.987145
3,0.166900,0.110447,0.961670,0.977036,0.973976,0.980116,4042,82,108,725,0.987520


[I 2025-05-05 22:09:41,372] Trial 3 finished with value: 0.9770364998791394 and parameters: {'learning_rate': 1.4369789613249647e-05, 'weight_decay': 0.009247035100297606, 'warmup_ratio': 0.06658598243129632, 'batch_size': 16}. Best is trial 0 with value: 0.9783581187280861.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,True Positives,False Negatives,False Positives,True Negatives,Roc Auc
1,No log,0.113466,0.962477,0.977405,0.979309,0.975509,4023,101,85,748,0.986718


[W 2025-05-05 22:23:35,456] Trial 4 failed with parameters: {'learning_rate': 3.5880789488061963e-05, 'weight_decay': 0.03608788941516705, 'warmup_ratio': 0.1201761323619273, 'batch_size': 16} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/nikhilghosh/Documents/Visual Studio Code/Courses/cse3000-ethics/.venv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/h6/86cprf7s3kxg1526bnp6tn0w0000gn/T/ipykernel_69869/119526113.py", line 200, in objective
    trainer.train()
  File "/Users/nikhilghosh/Documents/Visual Studio Code/Courses/cse3000-ethics/.venv/lib/python3.10/site-packages/transformers/trainer.py", line 2241, in train
    return inner_training_loop(
  File "/Users/nikhilghosh/Documents/Visual Studio Code/Courses/cse3000-ethics/.venv/lib/python3.10/site-packages/transformers/trainer.py", line 2553, in _inner_training_loop
    and (torch.isnan(

KeyboardInterrupt: 

In [ ]:
# load the best model and tokenizer
avg_results = {
    metric: np.mean([fold[f"eval_{metric}"] for fold in fold_results])
    for metric in fold_results[0].keys() if metric.startswith("eval_")
}

print("\nAverage results across folds:")
for metric, value in avg_results.items():
    print(f"{metric}: {value:.4f}")

# Train final model on all data
print("\nTraining final model on all data...")


In [ ]:
# Save optimal threshold in model config
# model.config.threshold = optimal_threshold
model.save_pretrained(MODEL_DIR)

# Example usage
sample_tweets = [
    "I love this new app! It's amazing!",
    "You are so stupid, I hate you",
    "The weather is nice today :)",
    "@user This is completely unacceptable behavior #angry"
]

print("\nTesting final model on sample tweets:")
for tweet in sample_tweets:
    result = predict_with_threshold(model, tokenizer, tweet, threshold=optimal_threshold)
    print(f"Tweet: {tweet}")
    print(f"Prediction: {result['class_name']} (confidence: {result['confidence']:.4f})")
    print(f"Offensive probability: {result['offensive_probability']:.4f}")
    print()